# Human heart failure

In this notebook, we show how to infer cell-level spatial transcriptome and how to perform analyses on the tissue samples from human heart failure patients with myocardial infarction.

* Thor inference
* Pathway enrichment analysis
* Transcription factor analyses
* Semi-supervised annotation of fibrotic cells (link to external video tutorials to Mjolnir platform, which is to be launched)

The input data and Thor processed data can be downloaded from our [google drive](https://drive.google.com/drive/folders/10k-bDw2zz_3Tdd9OvcSjNNqfKmTNbKqn?usp=sharing) or from the original [Nature paper](https://www.nature.com/articles/s41586-022-05060-x). Aliases (in parentesis) are used in our manuscript of the sample names appeared in the original paper: RZ_GT_P2(RZ1), RZ_P11(RZ2), GT_IZ_P9(IZ1), IZ_P10(IZ2), FZ_GT_P19(FZ1), FZ_P18(FZ2).

For installation of Thor, please refer to [this installation guide](../installation.rst).

## Import the packages

In [ ]:
import os
import logging
import datetime
import numpy as np

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(name)s - %(levelname)s - %(message)s')

now = datetime.datetime.now()
logger.info(f"Current Time: {now}")

In [2]:
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import scanpy as sc
import decoupler as dc 
import warnings

warnings.filterwarnings("ignore")
sc.set_figure_params(scanpy=True, dpi=150, dpi_save=300)
sc.settings.verbosity = 'error'

from thor.analy import get_pathway_score, get_tf_activity
from thor.finest import fineST
from thor.pl import colors 

## Predicting cell-level gene expression using Markov graph diffusion 
There are in total of six samples. We use sample "RZ_GT_P2" as an exmaple. All the process and parameters were kept the same.

We also skip the preprocessing of the images and the spatial transcriptome data in this tutorial. Please refer to the other two tutorials for preprocessing.
The preprocessed data can be downloaded from the same google drive [link](https://drive.google.com/drive/folders/1TqDRuqOd6I5RIKN-G-Nu0E2SpORX4BCH?usp=sharing).

In [3]:
samples = ['RZ_GT_P2', 'RZ_P11', 'GT_IZ_P9', 'IZ_P10', 'FZ_GT_P19', 'FZ_P18']

library_names = {
    'FZ_P20': 'Visium_16_CK294',
    'FZ_GT_P19': 'Visium_14_CK292',
    'GT_IZ_P9': 'Visium_9_CK287',
    'IZ_P10': 'Visium_7_CK285',
    'RZ_P11': 'Visium_8_CK286',
    'RZ_GT_P2': 'Visium_5_CK283'
    }

In [ ]:
sn = samples[0]

# Below are the files you needed to run Thor
image_path = os.path.join("HE_image", sn, f"{library_names[sn]}.tif")
cell_features_csv_path = os.path.join("WSI_HF", sn, f"features_{sn}.csv")
cell_mask_path = os.path.join("WSI_HF", sn, f"nucleis_{sn}_mask.npz")
spot_adata_path = os.path.join("Spatial_HF", sn, f"{sn}_adata_spots_thor_processed.h5ad")

rz = fineST(
    image_path, 
    sn, 
    spot_adata_path=spot_adata_path,
    cell_features_csv_path=cell_features_csv_path,
    recipe='gene'
    
    )
rz.prepare_input(mapping_margin=10)


In [ ]:
rz.adata

Run time ~ 10 mins for 32994 cells and 14467 genes on a MacbookPro M2 laptop.

In [ ]:
rz.set_genes_for_prediction(genes_selection_key='all')

rz.set_params(
    out_prefix="fineST",
    is_rawCounts=False,
    layer=None,
    is_rawCount=True,
    write_freq=20,
    n_iter=20,
    smoothing_scale=0.8,
    node_features_obs_list=['spot_heterogeneity'],
    n_neighbors=5,
    geom_morph_ratio=1,
    inflation_percentage=None,
    regulate_expression_mean=False,
    stochastic_expression_neighbors_level=None,
    n_jobs=8)

rz.predict_gene_expression()


In [ ]:
ad_thor = rz.load_result("fineST_20.npz")
sc.pp.normalize_total(ad_thor)
sc.pp.log1p(ad_thor)
ad_thor

In [8]:
palette = colors.continuous_palettes['blueYellow']
cmap = LinearSegmentedColormap.from_list('my_cmap', palette, N=256)

In [ ]:
sc.pl.spatial(ad_thor, color=['PDGFRA', 'CASQ2', 'FBLN2', 'PKP2'], spot_size=40, use_raw=False, ncols=4, cmap=cmap, vmin='p1', vmax='p99')

## Pathway enrichment analyses

Get the gene sets for the GO_BP pathways

In [ ]:
import decoupler as dc

msigdb = dc.get_resource('MSigDB')
msigdb = msigdb[msigdb['collection'] == 'go_biological_process']
msigdb = msigdb[~msigdb.duplicated(['geneset', 'genesymbol'])]
msigdb['geneset'] = [name.split('GOBP_')[1] for name in msigdb['geneset']]
msigdb

In [ ]:
ptw_list = [
    'POSITIVE_REGULATION_OF_T_CELL_PROLIFERATION',
    'POSITIVE_REGULATION_OF_INFLAMMATORY_RESPONSE',
    'POSITIVE_REGULATION_OF_IMMUNE_EFFECTOR_PROCESS',
    'POSITIVE_REGULATION_OF_B_CELL_ACTIVATION',
    'POSITIVE_REGULATION_OF_FIBROBLAST_PROLIFERATION',
    'COLLAGEN_FIBRIL_ORGANIZATION',
    'CARDIAC_MUSCLE_CONTRACTION'
    ]
msigdb = msigdb[msigdb.geneset.isin(ptw_list)]
msigdb

Calculate the enrichment score using decoupler

In [ ]:
acts = get_pathway_score(ad_thor, layer=None, net_df=msigdb)
acts

The pathway enrichment scores can be visualized on our interactive browser Mjolnir or using SCANPY as follows

In [ ]:
for pathway in ptw_list:
    sc.pl.spatial(acts, color=pathway, layer='smoothed', spot_size=30, use_raw=False, ncols=3, cmap='rainbow', frameon=False, title=pathway.lower(), colorbar_loc=None, alpha=0.5, alpha_img=0.7)

## Transcription factor activity inference


Download the CollecTRI database.

In [ ]:
net = dc.get_collectri(organism='human', split_complexes=False)
net

Calculate the TF activity based on regulated gene expression using decoupler

In [ ]:
acts = get_tf_activity(ad_thor, layer=None, net_df=net)
acts

The TF activity score can be visualized on our interactive browser Mjolnir or using SCANPY as follows

In [ ]:
tf_list = ['MYB', 'STAT4', 'SMAD3']
for tf in tf_list:
    sc.pl.spatial(acts, color=tf, layer='smoothed', spot_size=30, use_raw=False, ncols=3, cmap='rainbow', frameon=False, title=tf, alpha=0.5, alpha_img=0.7)

## Semi-supervised annotation

The semi-supervised annotation of similar cells can be performed on our web-based platform Mjolnir. Below are three video tips for how to use mjolnir.

- [Visualization of gene expression at spot and cell level](https://drive.google.com/file/d/1g_zGoHohbeXP2RFN_AVAhTQqSAKY5qcj/view?usp=sharing)
- [Visualization of pathway enrichment heatmaps](https://drive.google.com/file/d/1e04zKErfZnXknPB3U7eENIfyq_o-RaBS/view?usp=sharing)
- [Semi-supervised annotation of cells](https://drive.google.com/file/d/1gZYKZTNUnzS_qd2YZrGEoWd7RYhoIAjd/view?usp=sharing)